# Heatmap CHIRPS Wilayah Lembang – Juni 2020 (dengan Titik Min/Max)
Visualisasi spasial curah hujan (CHIRPS) bulan Juni 2020 di wilayah Lembang menggunakan interpolasi linier dan masking poligon.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.path as mpath


## 1. Load Data

In [ ]:

df = pd.read_csv("Lembang_Merged_SPI_CLEANED.csv")
boundary_df = pd.read_csv("Koordinat_Polygon_Lembang.csv")


## 2. Filter Data CHIRPS Bulan Juni 2020

In [ ]:

df["date"] = pd.to_datetime(df["date"])
chirps_jun = df[df["date"] == "2020-06-01"]
chirps_jun = chirps_jun.groupby(["lat", "lon"])["CHIRPS"].mean().reset_index()


## 3. Interpolasi dan Masking Wilayah

In [ ]:

grid_lon = np.linspace(chirps_jun["lon"].min(), chirps_jun["lon"].max(), 300)
grid_lat = np.linspace(chirps_jun["lat"].min(), chirps_jun["lat"].max(), 300)
grid_lon, grid_lat = np.meshgrid(grid_lon, grid_lat)
grid_z = griddata((chirps_jun["lon"], chirps_jun["lat"]), chirps_jun["CHIRPS"], (grid_lon, grid_lat), method='linear')

boundary_coords = boundary_df[['lon', 'lat']].values
polygon_path = mpath.Path(boundary_coords)
points = np.vstack((grid_lon.flatten(), grid_lat.flatten())).T
mask = polygon_path.contains_points(points).reshape(grid_lon.shape)
masked_grid_z = np.where(mask, grid_z, np.nan)


## 4. Visualisasi Heatmap CHIRPS dengan Titik Min/Max

In [ ]:

min_val = chirps_jun["CHIRPS"].min()
max_val = chirps_jun["CHIRPS"].max()
min_loc = chirps_jun[chirps_jun["CHIRPS"] == min_val][["lon", "lat"]].values[0]
max_loc = chirps_jun[chirps_jun["CHIRPS"] == max_val][["lon", "lat"]].values[0]

plt.figure(figsize=(10, 10))
contour = plt.contourf(
    grid_lon, grid_lat, masked_grid_z,
    levels=100, cmap="YlGnBu", alpha=0.95
)
plt.colorbar(contour, label="CHIRPS (mm)")
plt.contour(grid_lon, grid_lat, masked_grid_z, levels=15, linewidths=0.4, colors='black', alpha=0.3)
plt.plot(boundary_coords[:, 0], boundary_coords[:, 1], color="white", linewidth=1.5)

plt.plot(*min_loc, marker='o', color='black')
plt.text(min_loc[0], min_loc[1], f"Min: {min_val:.2f}", fontsize=10, ha='right', va='bottom', color='black')

plt.plot(*max_loc, marker='o', color='black')
plt.text(max_loc[0], max_loc[1], f"Max: {max_val:.2f}", fontsize=10, ha='left', va='top', color='black')

plt.grid(True, which='major', linestyle='--', linewidth=0.5, alpha=0.4)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Peta Sebaran Interpolatif CHIRPS – Juni 2020 (dengan Titik Min/Max)", fontsize=13)
plt.gca().set_facecolor("white")
plt.axis("equal")
plt.tight_layout()
plt.show()
